In [ ]:
import csv

# This function reads a CSV file and returns a dictionary with the ID as the key and the name, latitude, and longitude as the value
def read_csv_to_dict(file_path):
    data_dict = {}
    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip the header row
        for row in csv_reader:
            id = row[6]
            name = row[7]
            latitude = row[1]
            longitude = row[2]
            data_dict[id] = {'name': name, 'latitude': latitude, 'longitude': longitude}
    return data_dict

file_path = 'erratics_gis.csv'  # Replace with your actual CSV file path
result = read_csv_to_dict(file_path)
print(result)


In [2]:
# Import modules
from arcgis import GIS
from arcgis.geometry import Point, Polyline, Polygon
from arcgis.map.symbols import SimpleMarkerSymbolEsriSMS, SimpleLineSymbolEsriSLS, SimpleMarkerSymbolStyle, PictureMarkerSymbolEsriPMS
from arcgis.map.popups import PopupInfo

In [3]:
# Log in to portal
portal = GIS()  # anonymously

In [4]:
# Create a list of point geometries and attributes
point_geometries = []
point_attributes = []
for id, data in result.items():
    name = data['name']
    latitude = data['latitude']
    longitude = data['longitude']
    point = Point({'x': longitude, 'y': latitude, 'spatialReference': {'wkid': 4326}})
    point_geometries.append(point)
    point_attributes.append({'name': name, 'description': id})

In [5]:
# Create a list of point graphics
point_symbol =  SimpleMarkerSymbolEsriSMS(
    style = SimpleMarkerSymbolStyle.esri_sms_circle,
    color = [0, 0, 0],
    outline = SimpleLineSymbolEsriSLS(color = [255, 255, 255], width = 1),
)

image_symbol = PictureMarkerSymbolEsriPMS(
    url = "https://upload.wikimedia.org/wikipedia/en/a/ab/Nevertry.jpg",
    width = 20,
    height = 20
)

In [ ]:
map = portal.map()
map

In [7]:
# Add the point graphics to the map
for point_geometry, point_attributes in zip(point_geometries, point_attributes):
    map.content.draw(
        shape=point_geometry,
        # symbol=point_symbol,
        symbol=image_symbol,
        popup=PopupInfo(title=point_attributes["name"], description=point_attributes["description"])
    )

In [8]:
# Set the map's extent
map.center = [34.0122, -118.8055]
map.zoom = 14

In [ ]:
# Export the map to an HTML file
import os

file_dir = os.path.join(os.getcwd(), "home")
if not os.path.isdir(file_dir):
    os.mkdir(file_dir)

file_path = os.path.join(file_dir, "add-a-point-line-and-polygon.html")

map.export_to_html(file_path, title="Add a point, line, and polygon")